**GRUPO 2**

> **Integrantes**:
>
> - Roberto Perez Esteban
> - Sofia Gomez Ramirez
> - Agustin Arganin Castillo

Proyecto de análisis exploratorio de datos de inmuebles

In [1]:
import csv
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import scipy
import seaborn as sns

# Proyecto EDA
## Análisis de Viviendas

In [ ]:
path = "../data"

In [ ]:
dirs = os.listdir(path)

In [38]:
df_2023_05_20 = pd.read_csv(f"{path}/23-05-20/DatosDescarga.csv")
df_2023_05_20.sample(5)

,Unnamed: 0,propertyCode,price,propertyType,operation,size,exterior,rooms,bathrooms,address,...,newDevelopment.1,priceByArea,detailedType,suggestedTexts,hasPlan,topNewDevelopment,superTopHighlight,floor,hasLift,parkingSpace
626,626,97714097,1125000.0,flat,sale,151.0,True,2,2,barrio Nueva España,...,False,7450.0,{'typology': 'flat'},"{'subtitle': 'Nueva España, Madrid', 'title': ...",True,False,False,6,True,"{'hasParkingSpace': True, 'isParkingSpaceInclu..."
793,793,99591610,354223.0,flat,sale,71.0,True,1,1,"Calle del Teniente Cnel. Noreña, 22",...,True,4989.0,{'typology': 'flat'},"{'subtitle': 'Legazpi, Madrid', 'title': 'Piso...",True,False,False,3,True,"{'hasParkingSpace': True, 'isParkingSpaceInclu..."
555,555,100112527,236000.0,flat,sale,69.0,False,2,1,Plaza Lavapiés,...,False,3420.0,{'typology': 'flat'},"{'subtitle': 'Lavapiés-Embajadores, Madrid', '...",True,False,False,1,False,NaN
699,699,99858104,5000000.0,flat,sale,400.0,True,4,5,Calle de Velázquez s/n,...,False,12500.0,{'typology': 'flat'},"{'subtitle': 'Recoletos, Madrid', 'title': 'Pi...",True,False,False,4,True,"{'hasParkingSpace': True, 'isParkingSpaceInclu..."
138,138,99302791,465000.0,flat,sale,94.0,True,2,2,Calle de Muñopedro,...,False,4947.0,{'typology': 'flat'},"{'subtitle': 'Imperial, Madrid', 'title': 'Pis...",True,False,False,1,True,NaN


In [31]:
df_2023_05_20.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1192 entries, 0 to 1191
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   latitude   1192 non-null   float64
 1   longitude  1192 non-null   float64
dtypes: float64(2)
memory usage: 18.8 KB


In [7]:
df_2023_05_20.columns

Index(['Unnamed: 0', 'propertyCode', 'price', 'propertyType', 'operation',
       'size', 'exterior', 'rooms', 'bathrooms', 'address', 'province',
       'municipality', 'district', 'country', 'newDevelopment', 'latitude',
       'longitude', 'showAddress', 'url', 'description', 'status',
       'newDevelopment.1', 'priceByArea', 'detailedType', 'suggestedTexts',
       'hasPlan', 'topNewDevelopment', 'superTopHighlight', 'floor', 'hasLift',
       'parkingSpace'],
      dtype='object')

In [10]:
df_2023_05_20["status"].value_counts()

status
good              896
renew             260
newdevelopment     40
Name: count, dtype: int64

In [12]:
df_2023_05_20["superTopHighlight"].value_counts()
df_2023_05_20["superTopHighlight"].dtype

dtype('bool')

In [15]:
df_2023_05_20.loc[:4, "suggestedTexts"].values

array(["{'subtitle': 'Peñagrande, Madrid', 'title': 'Piso'}",
       "{'subtitle': 'Peñagrande, Madrid', 'title': 'Ático'}",
       "{'subtitle': 'Peñagrande, Madrid', 'title': 'Piso en Calle de Cándido Mateos'}",
       "{'subtitle': 'Peñagrande, Madrid', 'title': 'Piso'}",
       "{'subtitle': 'Peñagrande, Madrid', 'title': 'Piso en Calle de Juan Sánchez'}"],
      dtype=object)

In [ ]:
df_2023_05_20.loc[:4, "parkingSpace"].values

array(["{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': True}",
       "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': True}",
       "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': True}",
       nan,
       "{'hasParkingSpace': True, 'isParkingSpaceIncludedInPrice': True}"],
      dtype=object)

In [18]:
df_2023_05_20.loc[:4, ["propertyCode", "url"]].values

array([[97392656, 'https://www.idealista.com/inmueble/97392656/'],
       [100340472, 'https://www.idealista.com/inmueble/100340472/'],
       [101481291, 'https://www.idealista.com/inmueble/101481291/'],
       [101181100, 'https://www.idealista.com/inmueble/101181100/'],
       [101308619, 'https://www.idealista.com/inmueble/101308619/']],
      dtype=object)

In [19]:
df_2023_05_20["province"].value_counts()

province
Madrid    1196
Name: count, dtype: int64

#### 23-05-20 | 23-10-29

#### 23-12-1 | 24-01-08

#### 24-03-11 | 24-04-20